# Dual Numbers and Automatic Differentation

## Motivation


Dual numbers are used to compute automatic derivatives of polynomials and other functions without requiring you to find the derivative analytically. How is this done? What are dual numbers.


First, let's just see what this means. Consider the equation

$$\begin{align} f(x)=&(x+3)(x+2)\\
 =& x^2 + 5x+6\end{align}$$
 
From this we know the derivative is

$$f'(x) = 2x + 5$$


We could then write code








In [1]:
def f(x):
    return x**2 + 5*x + 6

def df(x):
    return 2*x + 5

And then evaluate it at a given value $x=3$

In [2]:
print(f'{f(3)=}')
print(f'{df(3)=}')

f(3)=30
df(3)=11


The rest of this notebook explains how to use dual numbers to find the derivative automatically; that is, you write the function `f(x)`, don't write `df(x)`, and yet the derivative (11 in the example above) is computed automatically for you.

# Dual Numbers

We don't want to derive the derivative, as it can be quite difficult or impossible. It is also a lot of boilerplate code that increases the chances of bugs. Wouldn't it be better if the computer could find it automatically? With dual numbers it can. 


## Hypercomplex and Complex Numbers

What is a dual number? Formally, it is a type of *hypercomplex numbers*. You are already familiar with some of these, even if you don't recognize the term. *Complex numbers* are another type of hypercomplex numbers, with the form

$$ a + bi$$

where $a$ and $b$ are real numbers, and $i^2 = -1$. We can write this more concisely as

$$a + bi\begin{cases}a,b \in \mathbb{R},i^2 = -1\end{cases}$$


Perhaps your mind fought this idea a bit when you first learned it, as no real number squared can equal -1. The great mathematician Euler had trouble with it. However, it is trivial to show they are utterly necessary when finding roots of cubic polynomials, even when x is in $\mathbb{R}$ (the real numbers). I refer you to chapter one of Needham's outstanding *Visual Complex Analysis* should you like a refresher on that. 

For now I will assume you are comfortable with complex numbers. We can do math on them:

$$\begin{align} x + y =& \\
   =& a + bi + c + di \\
   =& (a + c) +  (b+d)i\end{align}$$
   
So we add complex numbers by adding the real and complex components separately. Similar rules can be derived for subtraction, multiplication, division, exponentation, etc. 

So, hypercomplex numbers are numbers that take a form 

$$ a_0 + a_1 e_1 + a_2 e_2 + a_3 e_3 + ... + a_n, e_n$$

where (a_0,...a_n) are real coefficients, and (1,e_1,...e_n) form a basis for the algebra. As an aside, most literature uses *i*, not *e*, but in this context it does not necessarily imply the complex *i*, so I used *e* to remove that confusion. 

Complex numbers use *i* for the basis where $n=1$. Another hypercomplex number you may already know are quaternions, which are used to represent rotations in computer graphics, robotics, etc. Quaternions are a hypercomplex number with $n=3$ and the basis numbers $(1,1,1,1)$. The usual notation for quaternions is $(w, x, y, z)$. 


## Dual Numbers

We are now ready for the definition of a dual number. A dual number is in the form

  
$$a + b\epsilon\begin{cases}a,b \in \mathbb{R},\epsilon^2 = 0\end{cases}$$

At first blush it may appear that this is just a real number. If $\epsilon^2 = 0$, then surely $\epsilon = 0$, and the number is just equal to $a$. But this is not so. The basis is not required to be in $\mathbb{R}$. For example, consider the matrix below who's square is the zero matrix:

$$\begin{align}\mathbf{x} &= \begin{bmatrix} 0 & 1 \\ 0 & 0\end{bmatrix} \\
\mathbf{x}^2 &= \mathbf{0} \end{align}$$

However, with dual numbers $\epsilon$ is an *infinitesimal number*. Infinitesimals are not in $\mathbb{R}$, as they are a non-zero number that is smaller than any real number. I'll state without proof that the square of an infinitesimal is zero.

This gives us the already stated definition of a dual number

$$a + b\epsilon\begin{cases}a,b \in \mathbb{R},\epsilon^2 = 0\end{cases}$$

with the added understanding that $\epsilon$ is an infinitesimal.

For example $3 +4\epsilon$ is a dual number, as is $\pi -\frac{1}{3}\epsilon$

### Using Dual Numbers to Find Derivatives
Okay, infinitesimals are used in calculus to find derivatives. Dual numbers use infinitesimals. We are looking for a way to automatically compute derivatives. This sounds promising!

What I provide now is not a full proof, but it takes you part of the way there, enough to understand what is happening. Consider the real polynomial

$$f(x) = p + qx + rx^2$$

Let's take it into the dual domain by replacing $x$ with $x + d\epsilon$:

$$\begin{align}
f(x + d\epsilon) &= p + q(x + d\epsilon) + r(x + d\epsilon)^2
\end{align}$$

Now we can refactor it as follows:

$$\begin{align}
f(x + d\epsilon) &= \\
&=p + q(x + d\epsilon) + r(x + d\epsilon)^2 \\
&=p + qx + qd\epsilon + rx^2 + 2rdx\epsilon \\
&=(p + qx + rx^2) + (q + 2rx)d\epsilon \\
&=f(x) + (q + 2rx)d\epsilon  \\
&=f(x) + f'(x)d\epsilon  \\
\end{align}$$

So we have a remarkable result. $(q + 2rx)$ is the derivative of $f(x)$, and it appears as the real component of $\epsilon$.

It takes a bit more work to turn this into a proof that this works for any polynomial, but any standard text can take you through that. I'm more interested in how we can use this in code to automatically compute derivatives. 

The main objection is that this only works for polynomials. However, we can express transcendental as a Taylor series with enough terms to achieve accuracy equal to the smallest floating point number representable by *double*, hence we can safely use this for any equation you can express in code. 

Let's see that. This library is written in Pyhon, but it is easily done in C++ using templates, or with any other language you prefer.

In [3]:
from dataclasses import dataclass

@dataclass
class Dual:
    real : float       # real part
    dual : float = 0   # infitesimal part
        
    def __repr__(self): return f'{self.real} + {self.dual}ε'

In [4]:
Dual(3,1)

3 + 1ε

Okay, we have a class that implements a dual number, but it isn't very useful yet. Let's implement addition. Addition works as it does with complex numbers, add the real parts and infitesimal part separately.

In [5]:
@dataclass
class Dual:
    real : float       # real part
    dual : float = 0   # infitesimal part
        
    def __add__(self, y):
        return Dual(self.real + y.real, self.dual + y.dual)
    
    def __repr__(self): return f'{self.real} + {self.dual}ε'    

In [6]:
Dual(3,4) + Dual(-5,6) + Dual(7)

5 + 10ε

We can add the rest of the mathematical operators. Subtraction and multiplication are easy, division is a bit more complicated to derive, see the wikipedia article for the derivation https://en.wikipedia.org/wiki/Dual_number.

In [7]:
@dataclass
class Dual:
    real : float       # real part
    dual : float = 0   # infitesimal part
        
    def __add__(self, y):
        if not isinstance(y, Dual):
            y = Dual(y, 0)
        return Dual(self.real + y.real, self.dual + y.dual)
    
    def __sub__(self, y):
        if not isinstance(y, Dual):
            y = Dual(y, 0)        
        return Dual(self.real - y.real, self.dual - y.dual)
    
    def __mul__(self, y):
        if not isinstance(y, Dual):
            y = Dual(y, 0)        
        return Dual(self.real * y.real, (self.real * y.dual) + (self.dual * y.real))
    
    def __truediv__(self, y):
        if not isinstance(y, Dual):
            y = Dual(y, 0)        
        y_real_inv = 1 / y.real
        real_div = self.real * y_real_inv
        return Dual(real_div, (self.dual - real_div*y.dual) * y_real_inv)

    def __repr__(self):
        if self.dual >= 0:
            return f'{self.real} + {self.dual}ε'
        else: 
            # dual is negative, avoid +- notation
            return f'{self.real} + {-self.dual}ε'

In [8]:
Dual(1,2) * Dual(3,4)

3 + 10ε

In [9]:
Dual(4,3) / Dual(2, 7)

2.0 + 5.5ε

Okay, we have implemented dual numbers, how do we compute the derivative? It's pretty simple, just define your function, and then pass in the dual number $x + 1\epsilon$. We've already shown that the result will contain the value of the function evaluated at $x$ in the real part, and the derivative times the coefficient $d$ in the dual part. Here we set it to 1 so that the dual coefficient is just the derivative.

We will use the equation $(x+2)(x+3)$ from the beginning of the article, where $f(3)=30$ and $f'(x)=11$

In [10]:
def f(x):
    # I didn't implement __pow__, so use the factored form
    return (x + 2) * (x + 3)

f(Dual(3, 1))

30 + 11ε

As you can see, the real part 30 equals $f(3)$, and the dual part 11 equals $f'(3)$.

A full implementation of dual numbers for automatic derivaties requires us to write the implementation of $sqrt$, $cos$, and the rest of the math functions. However, that is not our goal of this notebook. This repository's dual.py provides a full implmentation of dual numbers.

For example consider $f(x) = 2x^3+log(x)$, which has the derivative $f'(x) = 6x^2 + \frac{1}{x}$

In [11]:
import dual
from dual import Dual

def f(x):
    return 2*x**3 + dual.log(x)

def df(x):
    return 6*x**2 + 1 / x

print(f(Dual(3, 1))) # compute f' automatically
print(f(3))  
print(df(3)) # compute f' manually to check

assert f(Dual(3, 1)).real == f(3)
assert f(Dual(3, 1)).dual == df(3)

55.09861228866811 + 54.333333333333336ε
55.09861228866811
54.333333333333336
